In [1]:
import pymysql as ps
import os
import random 
import string
import pandas as pd
import datetime as dt
import warnings 

warnings.filterwarnings("ignore")

In [2]:
dbcon = ps.connect(host='localhost', user= 'root', password='Poorvi', database='Bank')

def dbconnection(q,data):
    cursor = dbcon.cursor()
        # Execute the insert query
    cursor.execute(q, data)
        # Commit the transaction
    dbcon.commit()

In [3]:
def main():
    print("------------------------------------------------------------------")
    print("                     BANK MANGEMENT SYSTEM                        ")
    print("------------------------------------------------------------------")
    option={1:'New Customer',2:'Existing Customer',3:'Exit'}
    print("Please choose any one option")
    for i,j in option.items():
        print(i,". ",j)
    op=int(input("Enter the choice from 1 to 3 : "))
    services(op)
    
def again(d):
    b = input("Do you want to continue Y or N ? ")
    if b.lower() =='y':
        option1={1:'Check Balance',2:'Deposit',3:'Withdraw',4:'Exit'}
        for i,j in option1.items():
            print(i,". ",j)
        op1=int(input("Enter the choice : "))
        sub_service(op1,d) 
    elif b.lower()=='n':
        main()
    else:
        print("Please enter the valid option")
        again(d)

In [4]:

def services(n):
    if n ==1:
        new_customer()     
        
    if n==2:
        existing_customer()
    if n==3:
        dbcon.close
        exit()
    if n>3:
        print("Please enter the valid option")
        op=int(input("Enter the choice from 1 to 3 : "))
        services(op)

In [5]:
def new_customer():
    print("Enter the below information about new customer")
    ac_no = input("Account no.  :   ") 
    df = pd.read_sql_query("SELECT * FROM customer WHERE A_no = %s", con=dbcon, params=(ac_no,))
    if df.shape[0]!=0:
        print("Account no has already created")
        new_customer()
    pin = input("Enter the pin (atleast length of 4):  ")
    while True:
        if len(pin)<=3:
            pin = input("Enter the pin (atleast length of 4):  ")
        else:
            break
            
    name = input("Customer_name :   ")
    add = input("Address        :   ")

    while True:
        ph_no= int(input("Phone no. :   "))
        if ph_no>9999999999:
            print("Please enter the valid phone no(10 digit)")
        else:
            break
         
    ac_type= input("Account type:   ")       
    amt = int(input("Amount(Rs) :   "))
    insert_query = "INSERT INTO customer VALUES (%s, %s, %s,%s, %s, %s,%s)"
    data = (ac_no,pin,name,add,ph_no,ac_type,amt)
    
    dbconnection(insert_query,data)
    
    print("A/c has created") 
    ch = input("More account to create (Y/N)? : ")
    if ch.upper()=='Y':
        new_customer()
    else:
        main()

In [6]:
def existing_customer():
    no =input("Account no.  :   ")

    df = pd.read_sql_query("SELECT * FROM customer WHERE A_no = %s", con=dbcon, params=(no,))
    if df.shape[0]!=0:
        print("Record found. Customer name : ",df.loc[0,'_name'])
        option1={1:'Check Balance',2:'Deposit',3:'Withdraw',4:'Statement',5:'Exit'}
        for i,j in option1.items():
            print(i,". ",j)
        op1=int(input("Enter the choice : "))
        sub_service(op1,df)        
    else:
        print("Record not found")
        existing_customer()

In [7]:
def sub_service(m,d):
    global i
    amt = d.loc[0,'amt']
    if m ==1:
        print("Available Balance = Rs ",amt)
        again(d)
    if m==2:
        dep= int(input("Enter the amount to deposit = Rs "))
        amt += dep
        print("Deposite successful \n Available Balance = Rs ",amt,"\n------------------------------------------------------------------")
        d['amt']= amt
        update_query = "update customer set amt =%s where A_no = %s"
        data = (amt,d.iloc[0,0])
        Trans_no ='Tran_d'+''.join([random.choice(string.ascii_letters) for i in range(4)])
        insert_query = "INSERT INTO deposit VALUES (%s, %s, %s,%s)"
        data1 = (Trans_no,d.loc[0,'A_no'],dep,dt.date.today())
        dbconnection(update_query,data)
        
        dbconnection(insert_query,data1)
        
        again(d)
       
    if m==3:
        withdraw(amt,d)
        again(d)
        
    if m==4:
        dep_df = pd.read_sql_query("SELECT * FROM deposit WHERE A_no = %s", con=dbcon, params=(d.loc[0,'A_no'],))
        dep_df['amount']=dep_df.loc[0,'d_amt']
        dep_df.drop('d_amt',axis=1,inplace=True)
        with_df= pd.read_sql_query("SELECT * FROM withdraw WHERE A_no = %s", con=dbcon, params=(d.loc[0,'A_no'],))
        with_df['amount']=0-with_df.loc[0,'w_amt']
        with_df.drop('w_amt',axis=1,inplace=True)
        full_df= pd.concat((dep_df, with_df), axis=0 )
        print("*"*30,"\n      Bank Statement      \n","*"*30)
        print(f"Account_no: {d.loc[0,'A_no']}\nCustomer name: {d.loc[0,'_name']}")
        print(full_df)
        
    if m==5:
        main()
   
    if m>5:
        print("Please enter the valid option")
        op2=int(input("Enter the choice : "))
        sub_service(op2,d)

In [8]:
def withdraw(amt,d):
    global j
    wid= int(input("Enter the amount withdraw = Rs "))
    if wid>amt:
        print(f"You do not have enough balance to withdraw,your current balance is {amt}")
        withdraw(amt,d)
    else:    
        amt -= wid
        print("Withdraw successful \n Available Balance = Rs ",amt,"\n------------------------------------------------------------------")
        d['amt']= amt
        update_query = "update customer set amt =%s where A_no = %s"
        data = (amt,d.iloc[0,0])
        Trans_no ='Tran_w'+''.join([random.choice(string.ascii_letters) for i in range(4)])
        insert_query = "INSERT INTO withdraw VALUES (%s, %s, %s,%s)"
        data1 = (Trans_no,d.loc[0,'A_no'],wid,dt.date.today())
        dbconnection(update_query,data)
        
        dbconnection(insert_query,data1)

In [9]:
main()

------------------------------------------------------------------
                     BANK MANGEMENT SYSTEM                        
------------------------------------------------------------------
Please choose any one option
1 .  New Customer
2 .  Existing Customer
3 .  Exit
Enter the choice from 1 to 3 : 1
Enter the below information about new customer
Account no.  :   102
Account no has already created
Enter the below information about new customer
Account no.  :   107
Enter the pin (atleast length of 4):  12345
Customer_name :   pooja
Address        :   surat
Phone no. :   6666666666
Account type:   saving
Amount(Rs) :   60000
A/c has created
More account to create (Y/N)? : n
------------------------------------------------------------------
                     BANK MANGEMENT SYSTEM                        
------------------------------------------------------------------
Please choose any one option
1 .  New Customer
2 .  Existing Customer
3 .  Exit
Enter the choice from 1 to

KeyboardInterrupt: Interrupted by user

In [1]:
dbcon.close()

NameError: name 'dbcon' is not defined